In [1]:
!unzip "/content/drive/MyDrive/영우_4기_딥러닝/dataset/cifar_10_small.zip" -d "/content/data"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: /content/data/train/cat/3_alley_cat_s_000009.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000010.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000031.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000044.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000048.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000080.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000083.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000085.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000088.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000117.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000118.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000128.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000169.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000181.png  
 extracting: /content/data/train/cat/3_alley_cat_s_000184.png  
 ext

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

In [8]:
train_dir = '/content/data/train'
test_dir = '/content/data/test'

In [9]:
#DataLoader
train_datagen = keras.preprocessing.image.ImageDataGenerator()
test_datagen = keras.preprocessing.image.ImageDataGenerator()

In [1]:
batch_size = 200
train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (128,128),
    batch_size = batch_size,
    interpolation = 'bilinear',
    color_mode = 'rgb',
    shuffle = 'True',
    class_mode = 'binary'
)
test_generator =test_datagen.flow_from_directory(
    directory = test_dir,
    target_size = (128,128),
    batch_size = batch_size,
    interpolation = 'bilinear',
    color_mode = 'rgb',
    shuffle = 'True',
    class_mode = 'binary'
)

NameError: ignored

In [11]:
augmenation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)
model = keras.Sequential([
    keras.Input(shape = (150,150,3)),
    augmenation,
    
    keras.layers.Conv2D(32,3, padding = 'same',activation = 'relu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),

    keras.layers.Conv2D(64,3, padding = 'same',activation = 'relu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
    

    keras.layers.Conv2D(128,3, padding = 'same',activation = 'relu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
     keras.layers.Conv2D(256,3, padding = 'same',activation = 'relu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
    
     keras.layers.Conv2D(512,3, padding = 'same',activation = 'relu',kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(),
    
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1,activation = 'sigmoid')
    ])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
batch_normalization_5 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 37, 37, 64)       

In [12]:
loss = keras.losses.binary_crossentropy
optim = keras.optimizers.Adam()
metric = keras.metrics.binary_accuracy

model.compile(
    loss = loss,
    optimizer = optim,
    metrics = [metric])



In [13]:
# 모델 최적화 설정
import os
MODEL_DIR = '/content/model_data'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath= MODEL_DIR + '/{epoch:02d}-{val_loss:.4f}.hdf5'
callback_list=[tf.keras.callbacks.ModelCheckpoint(filepath = modelpath, monitor = 'val_loss',verbose = 1,save_best_only= True),
               tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 10)]

In [19]:
train_aeroplane_dir = os.path.join(train_dir,'aeroplane')
train_bird_dir = os.path.join(train_dir,'bird')
train_car_dir = os.path.join(train_dir,'car')
train_cat_dir = os.path.join(train_dir,'cat')

number =  len(os.listdir(train_aeroplane_dir)) + len(os.listdir(train_bird_dir)) + len(os.listdir(train_car_dir)) + len(os.listdir(train_cat_dir))


history = model.fit(
    train_generator,
    steps_per_epoch = (number / batch_size),
    epochs = 1000,
    validation_data = test_generator,
    callbacks = callback_list,
    validation_freq = 1,
)

Epoch 1/1000
  16/1000 [..............................] - ETA: 19:41 - loss: -404.5545 - binary_accuracy: 0.2656

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
model = keras.models.load_model('/content/model_data/44-0.3398.hdf5')
model.summary()